In [208]:
from selenium.webdriver import Firefox
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.firefox.options import Options
import requests
from datetime import datetime
from bs4 import BeautifulSoup

In [209]:
# get total number of active power outages in Nova Scotia
def outages_all():
    url = "http://outagemap.nspower.ca/external/default.html"
    options = Options()
    options.add_argument('-headless')
    driver = Firefox(executable_path='geckodriver', options=options)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    return int([i.get_text().replace(",","") for i in soup.find_all("div","total-customers-affected-value")][0])

In [210]:
# returns a dictionary of regions and their number of outages
def outages_combined():
    url = "http://outagemap.nspower.ca/external/default.html"
    options = Options()
    options.add_argument('-headless')
    driver = Firefox(executable_path='geckodriver', options=options)
    driver.get(url)

    x1 = driver.find_element_by_xpath('//*[@id="menu-content"]/a[2]')
    x1.click()

    x2 = driver.find_element_by_xpath('//*[@id="view-summary-serviceArea"]')
    x2.click()

    driver.implicitly_wait(10)

    outs = {}
    #second last index: increase by 1 up to 34 
    count = 4
    for i in range(31):
        county = driver.find_element_by_xpath(f'//*[@id="report-panel-serviceArea-table"]/tbody/tr[{count}]/td[1]')
        outages = driver.find_element_by_xpath(f'//*[@id="report-panel-serviceArea-table"]/tbody/tr[{count}]/td[2]')
        outs[county.text] = outages.text
        count += 1

    del outs['']

    al = {}
    for k,v in outs.items():
        try:
            a = v.replace(',','')
            al[k] = int(a)
        except:
            al[k] = 5
    al['ALL'] = sum([v for k,v in al.items()])
    al['DATE'] = str(datetime.now())
    
    return al

In [211]:
outages_combined()

{'ALL': 12580,
 'ANTIGONISH': 240,
 'BRIDGEWATER': 3277,
 'CHESTER': 8,
 'CHETICAMP': 143,
 'DARTMOUTH': 1982,
 'DATE': '2020-02-27 22:35:32.293586',
 'HALIFAX': 2110,
 'LIVERPOOL': 46,
 'MABOU': 28,
 'PORT HAWKESBURY': 26,
 'RIVER BOURGEOIS': 1850,
 'SACKVILLE': 51,
 'SHEET HARBOUR': 160,
 "ST. MARGARET'S BAY": 2207,
 'STELLARTON': 308,
 'SYDNEY': 129,
 'TRURO': 5,
 'UPPER MUSQUODOBOIT': 5,
 'WINDSOR': 5}